In [52]:
import networkx as nx
import numpy as np
import re
G=nx.Graph()
nodes=[]
edge=[]
labelnames=[]
adjList={}
b=[]
file = open('football.gml', 'r')
for line in file.readlines():
    if re.search(' id ',line, re.I):
        nodes.append(int(line.replace("id","").strip()))
    if re.search('source',line, re.I):
        source=int(line.replace("source","").strip()) 
    if re.search('target',line, re.I):
        target= int(line.replace("target","").strip())  
        edge.append([source,target])
    if re.search('label',line, re.I): 
        labelnames.append(line.replace("label","").strip())
file.close()
G.add_nodes_from(nodes)
G.add_edges_from(edge)
for i in G.nodes():
    for j in G.nodes():
        if G.has_edge(i,j):
            b.append(j)
    adjList[i]=b
    b=[]


In [53]:
dicto={}
nodes_depth= {}
nodes_explored={}
depthLimit= 3

In [54]:
M = np.empty(shape=[0, len(nodes)])
for i in nodes:
    list1=[]
    openList=[]
    closeList=[]
    for j in nodes:
        dicto[j] = nx.clustering(G, j)
        nodes_explored[j]=False
    openList.append(i)
    nodes_depth[i] = 0
    while(openList):
        currNode= openList.pop()
        if(nodes_depth[currNode]==0):
            dicto[currNode] = float("inf")
        else:
            dicto[currNode] = dicto[currNode] + (nodes_depth[currNode] ** -2)
        for i in closeList:
            if(nodes_depth[i] >= nodes_depth[currNode]):
                nodes_explored[i]=False
                closeList.remove(i)
        if (nodes_depth[currNode] < depthLimit):
            for n in adjList[currNode]:
                if(not nodes_explored[n]):
                    openList.append(n)
                    nodes_depth[n] = nodes_depth[currNode] + 1
            nodes_explored[currNode] = True
            closeList.append(currNode)
    for ele in dicto:
        list1.append(dicto[ele])
    M = np.vstack([M,list1])
                    
                    
                

In [55]:
y = np.where(M==float("inf"), 0, M)

In [56]:
y

array([[0.        , 0.88383838, 0.48989899, ..., 0.51111111, 0.55555556,
        0.79545455],
       [1.34848485, 0.        , 0.37878788, ..., 0.65      , 0.55555556,
        0.65656566],
       [0.34848485, 0.55050505, 0.        , ..., 0.4       , 0.44444444,
        0.76767677],
       ...,
       [0.34848485, 0.43939394, 0.37878788, ..., 0.        , 0.69444444,
        0.54545455],
       [0.57070707, 0.43939394, 0.37878788, ..., 0.53888889, 0.        ,
        0.54545455],
       [0.57070707, 0.43939394, 0.6010101 , ..., 0.28888889, 0.55555556,
        0.        ]])

In [57]:
list2=[]

In [58]:
import pandas as pd
df = pd.DataFrame(
    [i, sum(y[i])] for i in range(0,len(nodes))
)

In [59]:
df[1]

0      67.688023
1      67.847114
2      69.018831
3      70.008730
4      60.618326
5      67.922872
6      80.301659
7      66.559235
8      54.008225
9      54.813781
10     68.869841
11     69.447619
12     68.730952
13     69.376912
14     55.380952
15     77.076912
16     62.673882
17     61.052165
18     63.164286
19     68.766811
20     66.561760
21     64.804185
22     59.610750
23     63.081962
24     73.897619
25     74.082973
26     69.758730
27     68.033983
28     66.703175
29     67.795599
         ...    
85     70.203175
86     67.922872
87     70.329942
88     75.276407
89     64.471861
90     61.508730
91     68.367316
92     77.681457
93     63.025397
94     69.158730
95     59.703175
96     60.864286
97     62.322222
98     69.172872
99     73.475397
100    66.969841
101    76.514286
102    67.614286
103    65.608730
104    71.950649
105    61.064286
106    75.442063
107    66.253175
108    59.653175
109    67.314791
110    69.916306
111    54.813781
112    59.8587

In [60]:
X=df.iloc[:,[0,1]].values

In [61]:
m= df.shape[0]
n= df.shape[1]

In [62]:
Centroids=np.array([]).reshape(n,0) 


In [63]:
Centroids

array([], shape=(2, 0), dtype=float64)

In [64]:
K=12

In [65]:
import random as rd
for i in range(K):
    rand=rd.randint(0,m-1)
    print(X[rand])
    Centroids=np.c_[Centroids,X[rand]]

[24.         73.89761905]
[13.         69.37691198]
[11.         69.44761905]
[65.        61.1450938]
[ 8.         54.00822511]
[103.          65.60873016]
[66.        64.8021645]
[36.         66.56031746]
[108.         59.6531746]
[ 3.         70.00873016]
[98.         69.17287157]
[ 4.         60.61832612]


In [66]:
EuclidianDistance=np.array([]).reshape(m,0)
for k in range(K):
    tempDist=np.sum((X-Centroids[:,k])**2,axis=1)
    EuclidianDistance=np.c_[EuclidianDistance,tempDist]
C=np.argmin(EuclidianDistance,axis=1)+1

In [67]:
n_iter=100


In [68]:
Y={}
for k in range(K):
    Y[k+1]=np.array([]).reshape(2,0)
for i in range(m):
    Y[C[i]]=np.c_[Y[C[i]],X[i]]
     
for k in range(K):
    Y[k+1]=Y[k+1].T
    
for k in range(K):
     Centroids[:,k]=np.mean(Y[k+1],axis=0)

In [69]:
for i in range(n_iter):
    EuclidianDistance=np.array([]).reshape(m,0)
    for k in range(K):
        tempDist=np.sum((X-Centroids[:,k])**2,axis=1)
        EuclidianDistance=np.c_[EuclidianDistance,tempDist]
    C=np.argmin(EuclidianDistance,axis=1)+1
     #step 2.b
    Y={}
    for k in range(K):
          Y[k+1]=np.array([]).reshape(2,0)
    for i in range(m):
          Y[C[i]]=np.c_[Y[C[i]],X[i]]
     
    for k in range(K):
          Y[k+1]=Y[k+1].T
    
    for k in range(K):
          Centroids[:,k]=np.mean(Y[k+1],axis=0)
    Output=Y

In [70]:
dict1={}

In [71]:
for k, v in Output.items():
    for i in range(len(v)):
        dict1[Output[k][i][0]] = k

In [72]:
import csv
with open('footballEdgeList.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Source", "Target"])
    for i in edge:
        writer.writerow([i[0], i[1]])
        

In [49]:
labelnames

['"BrighamYoung"',
 '"FloridaState"',
 '"Iowa"',
 '"KansasState"',
 '"NewMexico"',
 '"TexasTech"',
 '"PennState"',
 '"SouthernCalifornia"',
 '"ArizonaState"',
 '"SanDiegoState"',
 '"Baylor"',
 '"NorthTexas"',
 '"NorthernIllinois"',
 '"Northwestern"',
 '"WesternMichigan"',
 '"Wisconsin"',
 '"Wyoming"',
 '"Auburn"',
 '"Akron"',
 '"VirginiaTech"',
 '"Alabama"',
 '"UCLA"',
 '"Arizona"',
 '"Utah"',
 '"ArkansasState"',
 '"NorthCarolinaState"',
 '"BallState"',
 '"Florida"',
 '"BoiseState"',
 '"BostonCollege"',
 '"WestVirginia"',
 '"BowlingGreenState"',
 '"Michigan"',
 '"Virginia"',
 '"Buffalo"',
 '"Syracuse"',
 '"CentralFlorida"',
 '"GeorgiaTech"',
 '"CentralMichigan"',
 '"Purdue"',
 '"Colorado"',
 '"ColoradoState"',
 '"Connecticut"',
 '"EasternMichigan"',
 '"EastCarolina"',
 '"Duke"',
 '"FresnoState"',
 '"OhioState"',
 '"Houston"',
 '"Rice"',
 '"Idaho"',
 '"Washington"',
 '"Kansas"',
 '"SouthernMethodist"',
 '"Kent"',
 '"Pittsburgh"',
 '"Kentucky"',
 '"Louisville"',
 '"LouisianaTech"',
 '"Lo

In [50]:
import csv
with open('footballNodeList.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Id", "Label", "Influence centrality","community"])
    for i in nodes:
        writer.writerow([i, labelnames[i],df[1][i],dict1[i]])